In [1]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import MeanAbsoluteError
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam

# Set random seed
np.random.seed(0)

In [2]:
data = pd.read_csv('DeltaBS_data.csv')
data

,Company,Time,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,Net Debt,Price_Change
0,SIRI,2019-12-01,2976.0,1895.0,1081.0,68.1,0.25,-1438.0,-0.30,1458.0,1406.0,1.44
1,NEWT,2015-12-01,50.6,13.1,37.5,4.3,-2.25,34.0,-2.22,7.6,24.1,1.87
2,ADM,2019-12-01,3164.0,2926.0,238.0,-2.2,0.52,-1208.0,-2.08,1512.0,2654.0,5.38
3,PXD,2017-12-01,544.0,-324.0,868.0,0.5,4.94,872.0,4.96,-481.0,-25.0,-7.22
4,TROW,2018-12-01,153.9,106.5,47.4,-8.1,1.96,299.9,1.88,0.0,477.5,-12.61
5,PRU,2013-12-01,22546.0,25777.0,-3231.0,-2.0,-6.62,-3160.0,-6.49,3913.0,10574.0,38.89
6,LEG,2017-12-01,566.7,469.9,96.8,-3.5,0.84,63.8,0.49,291.9,47.7,-1.15
7,ADS,2012-12-01,3019.9,2667.4,352.5,-0.1,7.12,-148.4,-3.15,2387.1,1710.0,40.92
8,PPG,2018-12-01,-523.0,417.0,-940.0,-14.0,-2.49,-982.0,-4.28,870.0,1398.0,-14.59
9,VZ,2015-12-01,11559.0,7393.0,4166.0,-81.6,1.07,-9660.0,-2.82,-3699.0,2384.0,-0.56


In [3]:
prices = data['Price_Change'] 
data = data.drop(['Company', 'Time', 'Price_Change'], axis = 1)

col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Price_Change', prices)
data

Total Assets
Done
Total Liabilities
Done
Total Equity
Done
Total Shares Out. on Filing Date
Done
Book Value / Share
Done
Tangible Book Value
Done
Tangible Book Value Per Share
Done
Total Debt
Done
Net Debt
Done


,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,Net Debt,Price_Change
0,0.567761,0.674890,0.524838,0.209157,0.001012,0.478161,0.156527,0.660160,0.609477,1.44
1,0.558627,0.667503,0.518225,0.206439,0.000961,0.488115,0.156493,0.649944,0.600695,1.87
2,0.568348,0.678937,0.519496,0.206162,0.001017,0.479716,0.156496,0.660541,0.617408,5.38
3,0.560168,0.666180,0.523488,0.206277,0.001108,0.493781,0.156620,0.646502,0.600383,-7.22
4,0.558950,0.667870,0.518288,0.205911,0.001047,0.489913,0.156566,0.649890,0.603576,-12.61
5,0.628862,0.768633,0.497513,0.206171,0.000871,0.466516,0.156418,0.677453,0.667741,38.89
6,0.560238,0.669296,0.518601,0.206107,0.001024,0.488316,0.156541,0.651947,0.600845,-1.15
7,0.567898,0.677922,0.520221,0.206252,0.001152,0.486881,0.156477,0.666705,0.611409,40.92
8,0.556836,0.669089,0.512031,0.205659,0.000956,0.481244,0.156457,0.656019,0.609426,-14.59
9,0.594558,0.696471,0.544388,0.202780,0.001028,0.422563,0.156483,0.623835,0.615692,-0.56


In [4]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:9] # Parameters
y = NpMatrix[:,9] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))
number_of_features = len(X[1])

X: [[0.56776064 0.67489009 0.52483793 ... 0.15652723 0.66016046 0.60947678]
 [0.55862704 0.66750314 0.51822526 ... 0.1564935  0.649944   0.60069462]
 [0.56834761 0.67893704 0.51949583 ... 0.15649596 0.66054083 0.61740799]
 ...
 [0.56725797 0.67263699 0.52745512 ... 0.1565369  0.65582847 0.60690295]
 [0.5811891  0.69430837 0.52074422 ... 0.15652372 0.67007121 0.60685211]
 [0.56018282 0.67036623 0.51676141 ... 0.15630548 0.65148239 0.60218744]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 9
Total Number of Training instances: 4500


[  1.44   1.87   5.38 ...   5.83  77.77 334.81]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector 1.44
Total number of y values 4500


In [5]:
#Loss functions
# https://keras.io/api/losses/regression_losses/#meanabsoluteerror-class
# https://keras.io/api/losses/

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=9, activation='relu', input_shape=(number_of_features,)))
    
    #Add Dropout Layer
    network.add(layers.Dropout(0.2))
    
    network.add(layers.Dense(units=9, activation='relu', input_shape=(number_of_features,)))
    
    #Add Dropout Layer
    network.add(layers.Dropout(0.2))

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=1, activation='tanh'))

    # Compile neural network
    network.compile(#loss='mse', # Mean Squared Error
                    loss = MeanAbsoluteError(),
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['mae', 'mse']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [6]:
neural_network = KerasRegressor(build_fn=create_network, 
                                 epochs= 500, 
                                 batch_size= 5)

In [7]:
# Evaluate neural network using three-fold cross-validation
scores = cross_val_score(neural_network, X, y, cv=5)

Train on 3600 samples
Epoch 1/500
3600/3600 [==============================] - 3s 792us/sample - loss: 16.3251 - mae: 16.3251 - mse: 2887.9238
Epoch 2/500
3600/3600 [==============================] - 1s 364us/sample - loss: 16.2990 - mae: 16.2990 - mse: 2886.3572
Epoch 3/500
3600/3600 [==============================] - 1s 339us/sample - loss: 16.2975 - mae: 16.2975 - mse: 2886.2952
Epoch 4/500
3600/3600 [==============================] - 1s 334us/sample - loss: 16.2975 - mae: 16.2975 - mse: 2886.2869
Epoch 5/500
3600/3600 [==============================] - 1s 344us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2798
Epoch 6/500
3600/3600 [==============================] - 1s 356us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2739
Epoch 7/500
3600/3600 [==============================] - 1s 335us/sample - loss: 16.2974 - mae: 16.2974 - mse: 2886.2852
Epoch 8/500
3600/3600 [==============================] - 1s 335us/sample - loss: 16.2973 - mae: 16.2973 - mse: 2886.2737
Epoch 9/50

3600/3600 [==============================] - 2s 459us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2661
Epoch 69/500
3600/3600 [==============================] - 1s 401us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2676
Epoch 70/500
3600/3600 [==============================] - 1s 339us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2673
Epoch 71/500
3600/3600 [==============================] - 1s 354us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2676
Epoch 72/500
3600/3600 [==============================] - 1s 362us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2688
Epoch 73/500
3600/3600 [==============================] - 1s 390us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2659
Epoch 74/500
3600/3600 [==============================] - 1s 362us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2661
Epoch 75/500
3600/3600 [==============================] - 1s 349us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2664 - 
Epoch 76/500
3600/3600 [==========

3600/3600 [==============================] - 1s 382us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2642
Epoch 135/500
3600/3600 [==============================] - 1s 358us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2681
Epoch 136/500
3600/3600 [==============================] - 1s 365us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2681
Epoch 137/500
3600/3600 [==============================] - 1s 401us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2664
Epoch 138/500
3600/3600 [==============================] - 1s 415us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2654
Epoch 139/500
3600/3600 [==============================] - 1s 389us/sample - loss: 16.2973 - mae: 16.2973 - mse: 2886.2688
Epoch 140/500
3600/3600 [==============================] - 2s 433us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2695
Epoch 141/500
3600/3600 [==============================] - 2s 423us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2659
Epoch 142/500
3600/3600 [=====

3600/3600 [==============================] - 1s 392us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2666
Epoch 201/500
3600/3600 [==============================] - 1s 374us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2659
Epoch 202/500
3600/3600 [==============================] - 2s 425us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2673
Epoch 203/500
3600/3600 [==============================] - 1s 364us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2673
Epoch 204/500
3600/3600 [==============================] - 1s 407us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2642
Epoch 205/500
3600/3600 [==============================] - 1s 400us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2668
Epoch 206/500
3600/3600 [==============================] - 1s 384us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2646
Epoch 207/500
3600/3600 [==============================] - 1s 338us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2676
Epoch 208/500
3600/3600 [=====

3600/3600 [==============================] - 2s 446us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2681
Epoch 267/500
3600/3600 [==============================] - 2s 466us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2676
Epoch 268/500
3600/3600 [==============================] - 1s 357us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2666
Epoch 269/500
3600/3600 [==============================] - 1s 403us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2666
Epoch 270/500
3600/3600 [==============================] - 1s 334us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2666
Epoch 271/500
3600/3600 [==============================] - 1s 367us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2656
Epoch 272/500
3600/3600 [==============================] - 1s 330us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2673
Epoch 273/500
3600/3600 [==============================] - 1s 332us/sample - loss: 16.2972 - mae: 16.2972 - mse: 2886.2649
Epoch 274/500
3600/3600 [=====

...

3600/3600 [==============================] - 1s 395us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5024
Epoch 202/500
3600/3600 [==============================] - 1s 383us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5039
Epoch 203/500
3600/3600 [==============================] - 1s 382us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5022
Epoch 204/500
3600/3600 [==============================] - 1s 413us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5056
Epoch 205/500
3600/3600 [==============================] - 1s 381us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5022
Epoch 206/500
3600/3600 [==============================] - 1s 377us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5017
Epoch 207/500
3600/3600 [==============================] - 1s 379us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5049
Epoch 208/500
3600/3600 [==============================] - 2s 418us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5061
Epoch 209/500
3600/3600 [=====

3600/3600 [==============================] - 1s 403us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5015
Epoch 268/500
3600/3600 [==============================] - 1s 379us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5032
Epoch 269/500
3600/3600 [==============================] - 1s 382us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5032
Epoch 270/500
3600/3600 [==============================] - 1s 400us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5022
Epoch 271/500
3600/3600 [==============================] - 1s 393us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5027
Epoch 272/500
3600/3600 [==============================] - 1s 381us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5042
Epoch 273/500
3600/3600 [==============================] - 1s 379us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5046
Epoch 274/500
3600/3600 [==============================] - 2s 421us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5034
Epoch 275/500
3600/3600 [=====

3600/3600 [==============================] - 1s 413us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5054
Epoch 334/500
3600/3600 [==============================] - 1s 376us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5037
Epoch 335/500
3600/3600 [==============================] - 1s 381us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5024
Epoch 336/500
3600/3600 [==============================] - 1s 382us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5012
Epoch 337/500
3600/3600 [==============================] - 1s 413us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5032
Epoch 338/500
3600/3600 [==============================] - 1s 383us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5037
Epoch 339/500
3600/3600 [==============================] - 1s 382us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5054
Epoch 340/500
3600/3600 [==============================] - 1s 374us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5034
Epoch 341/500
3600/3600 [=====

3600/3600 [==============================] - 1s 264us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5024
Epoch 400/500
3600/3600 [==============================] - 1s 292us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5039
Epoch 401/500
3600/3600 [==============================] - 1s 262us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5017
Epoch 402/500
3600/3600 [==============================] - 1s 265us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5017
Epoch 403/500
3600/3600 [==============================] - 1s 264us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5039
Epoch 404/500
3600/3600 [==============================] - 1s 243us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5042
Epoch 405/500
3600/3600 [==============================] - 1s 244us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5034
Epoch 406/500
3600/3600 [==============================] - 1s 221us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5020
Epoch 407/500
3600/3600 [=====

3600/3600 [==============================] - 1s 213us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5015 - loss: 16.2071 - mae: 16.2071 - ms
Epoch 464/500
3600/3600 [==============================] - 1s 218us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5032
Epoch 465/500
3600/3600 [==============================] - 1s 211us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5046
Epoch 466/500
3600/3600 [==============================] - 1s 236us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5020
Epoch 467/500
3600/3600 [==============================] - 1s 217us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5017
Epoch 468/500
3600/3600 [==============================] - 1s 213us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659.5017
Epoch 469/500
3600/3600 [==============================] - 1s 210us/sample - loss: 15.9523 - mae: 15.9522 - mse: 2659.5020
Epoch 470/500
3600/3600 [==============================] - 1s 213us/sample - loss: 15.9523 - mae: 15.9523 - mse: 2659

In [9]:
print(scores)
print("Mean MAE:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

[-14.3344223  -16.60932206 -15.39902212 -17.46635584 -15.71408902]
Mean MAE: -15.90464226881663
Standard Deviation of the mean: 1.0670591283486894
